<a href="https://colab.research.google.com/github/piyuminadee/LangGraph/blob/main/langgraph_stage_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain==0.3.24 -q
!pip install -U langchain-google-genai
!pip install langgraph -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.5 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.7.0 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 18.4 MB/s eta 0:00:00


In [24]:
from typing import TypedDict, List, Union
from langchain_core.messages import HumanMessage, AIMessage

In [25]:
class AgentState(TypedDict):
  messages: List[HumanMessage]

Initialize OpenAI LLM

In [26]:
import os
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

In [27]:
# Set the Gemini API key from Colab secrets
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

# Initialize the ChatGoogleGenerativeAI model
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0
)

In [28]:
def chatbot(state: AgentState) -> AgentState:
  response = llm.invoke(state["messages"])
  print(f"\nAI: {response.content}")
  return state

In [29]:
from langgraph.graph import StateGraph, START, END
workflow = StateGraph(AgentState)
workflow.add_node("chatbot", chatbot)
workflow.add_edge(START, "chatbot")
workflow.add_edge("chatbot", END)
app = workflow.compile()

In [20]:
user_input = input("Enter a Query:")
result = app.invoke({"messages": [HumanMessage(content=user_input)]})
print(result)

Enter a Query:what is the capital of Canada

AI: Ottawa is the capital of Canada.
{'messages': [HumanMessage(content='what is the capital of Canada', additional_kwargs={}, response_metadata={})]}


To loop

In [ ]:
history = []
user_input = input("Enter a Query:")
while user_input != "exit":
   history.append(HumanMessage(content=user_input))
   result = app.invoke({"messages": history})
   print(result)
   history = result["messages"]
   user_input = input("Enter a Query: ")

Enter a Query:hi

AI: Hi there! How can I help you today?
{'messages': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={})]}
Enter a Query: im piyumi

AI: Hi Piyumi!  It's nice to meet you. How can I help you today?
{'messages': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), HumanMessage(content='im piyumi', additional_kwargs={}, response_metadata={})]}
Enter a Query: who am i

AI: You are a large language model, also known as a conversational AI or chatbot, trained to be informative and comprehensive.
{'messages': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), HumanMessage(content='im piyumi', additional_kwargs={}, response_metadata={}), HumanMessage(content='who am i', additional_kwargs={}, response_metadata={})]}
Enter a Query: no im piyumi

AI: You are Piyumi.  I am a large language model.
{'messages': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), HumanMessage(content='im piyumi', add